# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [3]:
# import required code
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import find_cells, segment_cells, foci
import os
# uncomment the following lines if you have jupyternotify installed and want to use it
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

/usr/local/lib/python3.6/site-packages/tifffile.py:313: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "
/usr/local/lib/python3.6/site-packages/czifile.py:130: UserWarning: failed to import the optional _czifile C extension module.
Decoding of JXR and JPEG encoded images will not be available.
Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _czifile C extension module.\n"
/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


<IPython.core.display.Javascript object>

The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [4]:
dATG7_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dAtg7_0hrTor_5Pos1_AiryscanProcessing.czi'
dATG7_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dAtg7_0hrTor_Empty_AiryscanProcessing.czi'
dFIP200_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dFIP200_0hrTor_5Pos_1_AiryscanProcessing.czi'
dFIP200_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dFIP200_0hrTor_Empty_AiryscanProcessing.czi'
WT_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_WT_0hrTor_5Pos1_AiryscanProcessing.czi'
WT_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_WT_0hrTor_Empty_AiryscanProcessing.czi'
dTMEM_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dTMEM_0hrTor_5Pos_1_AiryscanProcessing.czi'
dTMEM_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dTMEM_0hrTor_Empty_AiryscanProcessing.czi'
dVPS37A_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dVPS_1hrTor_20Pos_1_AiryscanProcessing.czi'
dVPS37A_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/wipi_atg16/WIPI-Atg16_HEK_dVPS_1hrTor_Empty_AiryscanProcessing.czi'
svm_pkl = '/Users/nweir/Dropbox/code/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [5]:
%%notify
## NOTE: RUNNING THIS CELL WILL TAKE A WHILE! DON'T BE CONCERNED! ##

# 1. load images and the trained SVM focus classifier, and 2.
dVPS37A_finder = find_cells.MultiFinder(filename=dVPS37A_czi,
                                         bg_filename=dVPS37A_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dATG7_finder = find_cells.MultiFinder(filename=dATG7_czi,
                                         bg_filename=dATG7_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dTMEM_finder = find_cells.MultiFinder(filename=dTMEM_czi,
                                         bg_filename=dTMEM_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dFIP200_finder = find_cells.MultiFinder(filename=dFIP200_czi,
                                         bg_filename=dFIP200_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
WT_finder = find_cells.MultiFinder(filename=WT_czi,
                                         bg_filename=WT_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
WT_splitter = segment_cells.CellSplitter(WT_finder)
dVPS37A_splitter = segment_cells.CellSplitter(dVPS37A_finder)
dATG7_splitter = segment_cells.CellSplitter(dATG7_finder)
dTMEM_splitter = segment_cells.CellSplitter(dTMEM_finder)
dFIP200_splitter = segment_cells.CellSplitter(dFIP200_finder)

# 3. identify nuclei
WT_splitter.segment_nuclei(verbose=True)
dVPS37A_splitter.segment_nuclei(verbose=True)
dATG7_splitter.segment_nuclei(verbose=True)
dTMEM_splitter.segment_nuclei(verbose=True)
dFIP200_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
WT_splitter.segment_cells(488, verbose=True)
dVPS37A_splitter.segment_cells(488, verbose=True)
dATG7_splitter.segment_cells(488, verbose=True)
dTMEM_splitter.segment_cells(488, verbose=True)
dFIP200_splitter.segment_cells(488, verbose=True)

# 5. Initialize foci detection objects.
dVPS37A_foci = foci.Foci(dVPS37A_splitter, verbose=True)
dATG7_foci = foci.Foci(dATG7_splitter, verbose=True)
dFIP200_foci = foci.Foci(dFIP200_splitter, verbose=True)
dTMEM_foci = foci.Foci(dTMEM_splitter, verbose=True)
WT_foci = foci.Foci(WT_splitter, verbose=True)

Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1

mask #3 complete.

generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
correcting slice labels...
One continuous stretch defined as in focus.
appending outputs...
mask #1 complete.

generating mask #2
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out o

passing segmented objs and seeds to Nuclei instance...

removing small perinuclear foci...
2 raw segmented DAPI foci
object volumes in pixels:
[401904 414813]
removing objects volume < 10000 px, #s:
[]
removing nuclei_centers corresponding to small DAPI foci...
2 raw segmented DAPI foci
object volumes in pixels:
[466948 214630]
removing objects volume < 10000 px, #s:
[]
removing nuclei_centers corresponding to small DAPI foci...
1 raw segmented DAPI foci
object volumes in pixels:
[580074]
removing objects volume < 10000 px, #s:
[]
removing nuclei_centers corresponding to small DAPI foci...
input image has 4 dimensions (multiple imgs)
segmenting image #1 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 4633
slice-by-slice cutoffs:
[   164.5    135.5    143.5    156.     175.     211.5    264.     303.
    361.     401.5    485.5    657.    1022.5   1656.    2499.5   3422.
   4455.    5642.5   7086.    8324.    9109.5   9684.   10318.5  10825.
  10932.5  10

watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #2 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 3221
slice-by-slice cutoffs:
[  144.    136.    169.5   188.    217.5   227.5   266.    301.5   430.
   665.    906.5  1117.5  1380.   1973.5  2996.5  4442.5  6025.   7145.
  7689.5  7928.5  8054.   7864.   7519.   7517.   7614.5  7500.   7282.
  7095.5  6783.   6391.   5452.   4409.5  3421.   2576.5  2099.5  1694.5
  1341.5  1204.   1067.5   982.    875.    734.    596.    472.    425.
   403.5   373.5   362.5   346.5   327.5   340. ]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei 

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell mask means:
[572.31175939761727, 592.58518463859502, 612.26501533590101, 630.4508931988081, 646.6239144077399, 660.79629172589534, 673.2509883352501, 684.44710920646082, 694.81194854463206, 704.53237953757434]
slopes:
[19.976627969141873, 18.93285428010654, 17.179449535919446, 15.17269926354362, 13.313536963755098, 11.825408740282739, 10.78048010469098, 10.04263516555676]
slope deltas:
[ 1.04377369  1.75340474  2.00675027  1.8591623   1.48812822  1.04492864
  0.73784494]
desired erosions: 4
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[570.57933116994548, 591.32047288494255, 611.67670114921896, 630.93496898258798, 648.74390725075216, 664.95223516102283, 679.81385073309139, 693.63234894690515, 706.57462529867678, 718.73035085288916]
slopes:
[20.54868498963674, 19.807248048822714, 18.533603050766601, 17.008633089217426, 15.534971741169613, 14.340056892941163, 13.380387282792697, 12.549000952992003]
slope deltas:
[ 0.74143694  1.273645    1.52496996  1.47366135  1.

removing small foci...
2 raw segmented cells
0 foci volume < 1000 px, removing...

segmenting cells in image #3 out of 3...
performing Euclidean distance xform...
smoothing the distance map...
generating segmentation seeds from nuclei_centers...
watershed segmenting cells from nuclei centers...
segmentation complete for image #3.
removing cells that contact the edge of the image.
removing small foci...
1 raw segmented cells
0 foci volume < 1000 px, removing...

checking that nuclei have already been segmented...
eroding cell edges...
eroding mask 0...
cell mask means:
[644.46621393219868, 671.45542653160248, 698.43256004331943, 724.12454410402029, 747.62057640515297, 768.54501407387352, 786.84791561904626, 802.76499705979245, 816.75882803476838, 829.19276205565757]
slopes:
[26.983173055560371, 26.334558786208902, 24.594008180916774, 22.210234984926615, 19.613669606946644, 17.109991492959466, 14.955456207861062, 13.213882497932559]
slope deltas:
[ 0.64861427  1.74055061  2.3837732   2.5

<IPython.core.display.Javascript object>

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.

A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [7]:
from skimage import io  # for saving images
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'  # change this to where you'd like the output images to go.
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dATG7_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dAtg7_raw_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.imgs[c][im, :, :, :])
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dVPS37A_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_raw_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.imgs[c][im, :, :, :])    
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dTMEM_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_raw_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.imgs[c][im, :, :, :])
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dFIP200_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_raw_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.imgs[c][im, :, :, :])
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, WT_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'WT_raw_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.imgs[c][im, :, :, :])

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarnin

## The following cell is for running segmentation on both channels.

In [ ]:
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (10000, 7500) 
red_thresholds = (10000, 7500)
# perform segmentation
dATG7_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dATG7_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dATG7_foci_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.foci[c][im])
dVPS37A_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dVPS37A_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_foci_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.foci[c][im])
dTMEM_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dTMEM_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_foci_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.foci[c][im])
dFIP200_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dFIP200_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_foci_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.foci[c][im])
WT_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(WT_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'WT_foci_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.foci[c][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (10000, 7500)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: nan
normalizing image...


/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.


/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:183: RuntimeWarning: invalid value encountered in greater
  eroded_mask = eroded_mask & (magnitude > 0)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:192: RuntimeWarning: invalid value encountered in greater_equal
  pts_plus = (isobel >= 0) & (jsobel >= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:193: RuntimeWarning: invalid value encountered in less_equal
  pts_minus = (isobel <= 0) & (jsobel <= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:193: RuntimeWarning: invalid value encountered in greater_equal
  pts_minus = (isobel <= 0) & (jsobel <= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:211: RuntimeWarning: invalid value encountered in greater_equal
  pts_plus = (isobel >= 0) & (jsobel >= 0) & (abs_isobel <= abs_jsobel)
/usr/local/lib/python3.6/sit

generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 0 foci in image


...

current ID: 32
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[103]
matched to one parent cell.
current ID: 33
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[48]
matched to one parent cell.
current ID: 34
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[256]
matched to one parent cell.
foci segmented from position 2

segmenting foci for position 3 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 2867.96951283
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 59 foci in image
cell me

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: Us

segmentation complete for channel 561
------------------------------------------------------
beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (10000, 7500)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 2918.52418541
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 108 foci in image
cell mean: 978.680227306
cell standard deviation: 1018.63162239
intensity cutoff for removal: 5562.52252805


current ID: 55
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[110]
matched to one parent cell.
current ID: 57
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[76]
matched to one parent cell.
current ID: 58
  overlapping parent cell(s): 
[1 3]
  # of pixels overlapping with parent(s):
[207  39]
  more than one overlapping cell.
  determining which cell overlaps more.
  parent cell: 1
current ID: 60
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[91]
matched to one parent cell.
current ID: 61
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[35]
matched to one parent cell.
current ID: 74
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[80]
matched to one parent cell.
current ID: 76
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[119]
matched to one parent cell.
current ID: 79
  overlapping parent cell(s): 
[2]
  #

focus intensity: 4981.0
removing focus
focus intensity: 4220.2
removing focus
focus intensity: 5239.03333333
removing focus
focus intensity: 5689.37313433
removing focus
focus intensity: 5383.13592233
removing focus
focus intensity: 4202.92592593
removing focus
focus intensity: 6712.97560976
focus intensity: 5823.50561798
removing focus
focus intensity: 5923.27586207
removing focus
focus intensity: 6010.82758621
removing focus
focus intensity: 7126.79487179
focus intensity: 5252.83783784
removing focus
after eliminating dim foci: 19 foci in image
eliminating foci that reside outside of cells...
eliminating intranuclear foci...
19 final foci
--matching foci to parent cells--
current ID: 4
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[111]
matched to one parent cell.
current ID: 8
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[32]
matched to one parent cell.
current ID: 16
  overlapping parent cell(s): 
[1]
  # of pixels over

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.

## The following cell is for running segmentation on just one channel.

In [ ]:
desired_channel = 488  # change this to 488 for green or 561 for red
# edit the (high, low) threshold pair below to optimize segmentation.
thresholds = (10000, 7500) 
# perform segmentation
neg_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
pos_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
# save segmentation outputs
for im in range(0, len(pos_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'pos_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             pos_foci.foci[desired_channel][im])
for im in range(0, len(neg_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'neg_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             neg_foci.foci[desired_channel][im])